
# Advisory mission for the Boston municipal authorities


![title](assets/boston_logo.png)

**Welcome to the Decision Science challenge of your data certification exam!**

⚠️ This challenge validates 2 blocks of the RNCP certification _Développeur en intelligence artificielle_ exam:  

**Block 1:**  *Analyse exploratoire et descriptive de la donnée*  
- 75% validated by Section 1 - Analysis of the mayor's team
- 25% validated by Section 2 - Analysis for the police department

**Block 2:**  *Développement et Gestion de projet en IA*  
- 100% validated by Section 2 - Analysis for the police department

To start, here is an overview of the `decision_science` directory:

In [1]:
# Start by running this cell to see the architecture of the directory
!tree

.
├── assets
│   └── boston_logo.png
├── boston_crimes.ipynb
├── data
│   ├── boston_crimes_regression.csv
│   ├── districts.csv
│   ├── incident_reports.csv
│   └── school.sqlite
├── data.py
├── db
│   └── boston_crimes.sqlite
├── Makefile
├── __pycache__
│   └── data.cpython-39.pyc
├── README.md
└── tests
    ├── answers_inference.json
    ├── __init__.py
    ├── linear_regression_analysis.pickle
    ├── linear_regression_model.pickle
    └── sql.pickle

5 directories, 16 files


- the `boston_crimes.ipynb` notebook that you currently have under your eyes is the main document. You will find all the instructions here and except when it is explicitly specified, you should provide all your answers in this notebook;


- the `data` and `db` folders will be filled-in throughout the exercise respectively with `.csv` datasets and a `.sqlite` file, for you to run all your analyses; 


- you will not have to interact with the `assets` folder for this exercise;


- the `tests` folder will contain all the `.pickle` files that will be saved throughout the exercise with your key findings. Please run all the "Save your results" cells when completing the exercise!

⚠️ **Important remark** before you dive into the exercise. This notebook is quite long and it is easy to get lost in it: take full advantage of the collapsible headers and of the table of content. If you have not yet activated these Jupyter Notebook extensions, you may consider doing so now!

# Imports

You can use this section to run your imports in a centralised manner throughout the exercise.

In [2]:
# Load the nbresult package to be able to save your results 
from nbresult import ChallengeResult

In [3]:
# Useful import for data collection
import sqlite3

In [4]:
# Useful imports for data manipulation and analysis
import numpy as np
import pandas as pd

In [5]:
# Useful imports for data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# Useful imports to estimate regression models
import statsmodels.formula.api as smf

# 1. Analysis for the mayor's team (4h)

**ℹ️ This part assesse the following skills of the first block of the certification _Analyse exploratoire et descriptive de la donnée_**:
- C1 Acquérir des données à partir de sources adaptées, fournies par le client ou accessibles en Open Data afin d'agrémenter la base de données
- C2 Écrire des requêtes pour l'interrogation de la base de données et les composants d'accès aux données
- C3 Effectuer des analyses exploratoires afin d'appréhender la composition interne des données

During the last municipal campaign in Boston, criminality has been a major topic of debates. As citizens have expressed strong expectations from her on that front, the newly-elected mayor of Boston is looking for data-based insights on criminality in the Massachussetts capital. She has mandated your economics and urbanism consulting firm, *The Locomotive*, for this study.

## 1.1 Load the database

_C1 Acquérir des données à partir de sources adaptées, fournies par le client ou accessibles en Open Data afin d'agrémenter la base de données_

Download the `boston_crimes.sqlite` database from this [URL](https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes.sqlite) and store it inside the `db` folder.

In [7]:
# You may directly run this cell to do so
#!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes.sqlite > db/boston_crimes.sqlite

## 1.2 Familiarize yourself with the database

This section aims at giving you a first overview of the database. 

As you will see, it consists in three tables: 

- the `incident_reports` table has been provided by the Police Department of Boston. Each observation corresponds to a criminal incident that has required an intervention by the police in the municipality of Boston;  



- the `districts` table has been provided by the Urbanism Department of Boston. It gathers geographical information about the various police districts of Boston;  



- and the `indicators` table has been shared by the Economics Department of Boston, which keeps track of various indicators of the social and economic activity of Boston neighborhoods. Each observation corresponds to a police district.

More information about the different fields of these three tables can be found in the dedicated `README.md` file

### Connect to the database

**🔍 Using your preferred SQL client, connect to the database and browse through it as you wish to get acquainted with the data.**

### Draw a schema of the database

**📝 Draw the database schema thanks to the [schema editor](https://kitt.lewagon.com/db) on Kitt.**

**📝 Download the schema and save it as `boston_crimes.xml` in the `db` folder.**

## 1.3 Extract the relevant dataset

_C2 Écrire des requêtes pour l'interrogation de la base de données et les composants d'accès aux données_

Now that you have a good overview of the database, you can kick off the work! You will start with an SQL query to gather the relevant information.

### Build the dataset

We want to investigate the influence of the socio-economic characteristics of Boston's different districts on the number of crime reports and incidents. To do so, we need to extract the relevant dataset. **Each row should correspond to one of the 12 police districts of Boston** (as listed in the `districts` table of the database).

To identify the district, we will need **the following columns**: 

- the `CODE` of the police district (1 letter and 1 or 2 numbers);
- the full `NAME` of the police district.

Additionally, you will need to **create an additional field** (which will serve as dependent variable in future regressions): `NB_INCIDENTS`, i.e. the total number of incidents reported in the police district over the period covered by the data at hand (2015-2019).

Eventually, we want the dataset to **include several socio-economic indicators**:

- `MEDIAN_AGE`;
- `TOTAL_POP`;
- `PERC_OF_30_34`;
- `PERC_MARRIED_COUPLE_FAMILY`;
- `PER_CAPITA_INCOME`;
- `PERC_OTHER_STATE_OR_ABROAD`;
- `PERC_LESS_THAN_HIGH_SCHOOL`;
- `PERC_COLLEGE_GRADUATES`. 

Overall, your dataset should comprise 12 rows and 11 columns.

Eventually, note that **the resulting DataFrame must be ordered by the number of incidents**, from the largest to the smallest total.

**📝 Write the SQL query you need to fetch the data. Save it as a `str` in the `query` variable.**

In [8]:
query = """
SELECT districts.CODE, NAME, INCIDENT_NUMBER, MEDIAN_AGE, TOTAL_POP, PERC_OF_30_34, PERC_MARRIED_COUPLE_FAMILY, 
PER_CAPITA_INCOME,PERC_OTHER_STATE_OR_ABROAD,PERC_LESS_THAN_HIGH_SCHOOL,PERC_COLLEGE_GRADUATES,
COUNT(NAME) AS NB_INCIDENTS
FROM districts
JOIN indicators ON districts.CODE = indicators.CODE
JOIN incident_reports ON districts.CODE = incident_reports.DISTRICT
GROUP BY NAME
ORDER BY NB_INCIDENTS
"""

**📝 Store the output of the query in a `DataFrame` named `crimes_df`. Display the 5 first rows, as well as the shape of the dataset.**

In [9]:
conn = sqlite3.connect('/home/demange/code/lewagon-assess/LDemange-data-boston-crimes-challenge-2022-07-12/db/boston_crimes.sqlite')
db = conn.cursor()
db.execute(query)
crimes_df = db.fetchall()
crimes_df = pd.DataFrame(crimes_df,columns=['districts', 'NAME', 'INCIDENT_NUMBER', 'MEDIAN_AGE', 'TOTAL_POP', 'PERC_OF_30_34', 'PERC_MARRIED_COUPLE_FAMILY', 
'PER_CAPITA_INCOME','PERC_OTHER_STATE_OR_ABROAD','PERC_LESS_THAN_HIGH_SCHOOL','PERC_COLLEGE_GRADUATES','NB_INCIDENTS'])
crimes_df[0:5]

,districts,NAME,INCIDENT_NUMBER,MEDIAN_AGE,TOTAL_POP,PERC_OF_30_34,PERC_MARRIED_COUPLE_FAMILY,PER_CAPITA_INCOME,PERC_OTHER_STATE_OR_ABROAD,PERC_LESS_THAN_HIGH_SCHOOL,PERC_COLLEGE_GRADUATES,NB_INCIDENTS
0,A15,Charlestown,I152049665,35.7,19890,28.2,36.4,75339,3.4,7.9,8.2,4765
1,E5,West Roxbury,I140036361-00,42.8,33526,18.1,46.2,47836,1.9,6.5,13.2,9406
2,A7,East Boston,I152049476,30.6,47263,31.1,30.4,31473,3.5,27.2,11.5,9691
3,E18,Hyde Park,I030217815-08,39.4,38924,21.1,38.4,32744,1.9,13.8,21.3,12551
4,E13,Jamaica Plain,I010370257-00,34.8,40867,32.5,33.7,51655,5.5,8.0,12.1,12802


In [10]:
print(crimes_df.shape)

(12, 12)


### 💾 Run the cell below to submit your results

In [11]:
from nbresult import ChallengeResult

result = ChallengeResult('sql', query=query, data=crimes_df)
result.write()

## 1.4 Linear regression - The socio-economic determinants of criminality

As mentioned above, we want to investigate the impact of the socio-economic characteristics of the different Boston police districts on the number of incidents that are reported in these areas. 
- We are going to use the number of incidents as dependent variable 
- our regressors will be the various socio-economic indicators extracted from the database.

*C3 Effectuer des analyses exploratoires afin d'appréhender la composition interne des données (50%)*

### 1.4.1 Start from a fresh dataset

To make sure that you are using the right data, you can load a fresh dataset from this [URL](https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes_regression.csv).

**📝 Load the data into a DataFrame named `data`**

In [12]:
data = pd.read_csv('/home/demange/code/lewagon-assess/LDemange-data-boston-crimes-challenge-2022-07-12/data/boston_crimes_regression.csv')
data

,MEDIAN_AGE,TOTAL_POP,PERC_OF_30_34,PERC_MARRIED_COUPLE_FAMILY,PER_CAPITA_INCOME,PERC_OTHER_STATE_OR_ABROAD,PERC_LESS_THAN_HIGH_SCHOOL,PERC_COLLEGE_GRADUATES,CODE,NB_INCIDENTS,NAME
0,30.8,55297,52.8,26.4,41261,8.6,6.7,10.5,D14,13788,Brighton
1,35.7,19890,28.2,36.4,75339,3.4,7.9,8.2,A15,4765,Charlestown
2,33.4,126909,28.2,26.6,29767,2.4,18.0,17.1,C11,32875,Dorchester
3,33.5,18306,32.5,35.8,80057,14.8,15.4,6.9,A1,26260,Downtown
4,30.6,47263,31.1,30.4,31473,3.5,27.2,11.5,A7,9691,East Boston
5,39.4,38924,21.1,38.4,32744,1.9,13.8,21.3,E18,12551,Hyde Park
6,34.8,40867,32.5,33.7,51655,5.5,8.0,12.1,E13,12802,Jamaica Plain
7,36.7,26659,20.9,29.8,28356,2.3,14.5,22.9,B3,28331,Mattapan
8,32.5,54161,27.8,17.8,20978,2.9,23.0,18.9,B2,38877,Roxbury
9,31.9,36772,46.1,24.7,64745,2.4,7.9,8.4,C6,16617,South Boston


### 1.4.2 Run the regression and output its summary

Thanks to the Statsmodels Formula API, we will run the regression described below. 

The dependent variable (or target variable) should be **the total number of incidents** reported in each police district.

We will focus on the following regressors: 

- the **median age** in the district, whose effect is difficult to anticipate on the number of crimes;
 
 
- the **percentage of 30-34 years old** in the district, whose effect is also unclear a priori;
 
 
- the **share of families with a married couple** among all households, which could be anticipated to have a negative effect on criminality (more attention to safety among residents...);
 
 
- the **percentage of residents having moved from abroad or from another US state over the last year**, mobility being often associated with social marginalisation and possibly with a higher risk of resorting to illegal activities;
 
 
- the **percentage of residents having stopped their studies before getting a high school degree**. Economic models would suggest that due to the more narrow job opportunities to which this group has access, the incentive is stronger to resort to illicit activities;
 
 
- the **percentage of college graduates** in the district, which we would expect to have an opposite effect.
 
**📝 Based on these indications, estimate the linear regression model and output its summary in this section of the notebook. Store the estimated model inside a `model` variable.**

In [13]:
from sklearn.linear_model import LinearRegression

import statsmodels.formula.api as smf
model = smf.ols('NB_INCIDENTS ~ MEDIAN_AGE + PERC_OF_30_34 + PERC_MARRIED_COUPLE_FAMILY + PERC_OTHER_STATE_OR_ABROAD + PERC_LESS_THAN_HIGH_SCHOOL + PERC_COLLEGE_GRADUATES', data=data).fit()
print(model.summary())

#Autre solution :
#X = data[['MEDIAN_AGE','PERC_OF_30_34','PERC_MARRIED_COUPLE_FAMILY','PERC_OTHER_STATE_OR_ABROAD','PERC_LESS_THAN_HIGH_SCHOOL','PERC_COLLEGE_GRADUATES']]
#y = data['NB_INCIDENTS']

# Instanciate model
#mlin = LinearRegression()
#mlin.fit(X, y)
#model = mlin.coef_
#model
#mlin.score


                            OLS Regression Results                            
Dep. Variable:           NB_INCIDENTS   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     7.391
Date:                Tue, 12 Jul 2022   Prob (F-statistic):             0.0222
Time:                        14:08:39   Log-Likelihood:                -114.59
No. Observations:                  12   AIC:                             243.2
Df Residuals:                       5   BIC:                             246.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

/home/demange/.pyenv/versions/lewagon/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


### 💾 Run the cell below to submit your results

In [14]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'linear_regression_model',
    data=data,
    model=model
)
result.write()

### 1.4.3 Statistical inference questions 🤯

*C3 Effectuer des analyses exploratoires afin d'appréhender la composition interne des données (50%)*

You will now be asked a series of statistical inference and methodological questions about the regression model estimated above. Don't worry if results do not perfectly fit the "predictions" that we made earlier about the impact of each regressor: the goal was to form an *a priori* hypothesis, which is validated or invalidated through this empirical analysis.

#### Questions on the results

**❓ Is the median age associated with a positive (increasing the number of crimes) or a negative (decreasing the target variable) effect on the number of crime incidents? Simply write your answer as a string below**

In [15]:
answer_median_age = '''Le coefficient lineaire estimé correspondant au MEDIAN_AGE est 2252.7344153, superieur a 0, on en 
déduit que le nombre de crimes augmente linéairement avec l age médian.
'''

**❓ What is the t-statistic associated with the median age regressor? How is it computed?**

💡 Hint: We are looking at a ratio

In [16]:
answer_t_statistic = ''' Le t statstique associé à l'âge median est 1.820. Il est calculé selon la fomule :
t =  (x-moy)/(sigma_echantillon/sqrt(N))
'''

**📝 Recompute approximately the t-statistic based on the regression summary.**

As it is a ratio $t = \frac{numerator}{denominator}$:
- Store the numerator into a `numerator` variable
- Store the denominator into a `denominator` variable
- Store the t-statistic into a `t_median_age` variable

In [17]:
numerator = '(x-moy)'
denominator =  'sigma_echantillon/sqrt(N)'
t_median_age = 1.820 #'(x-moy)/(sigma_echantillon/sqrt(N))'

**❓ What is the p-value associated with the median age regressor? Store it in the `pvalue_median_age` variable (you may directly copy-paste it from the regression summary).** 

In [18]:
pvalue_median_age = 0.834

**❓ What does this p-value mean for the median age? Is its effect statistically significant at the 95% confidence level? At the 90% confidence level? Simply formulate your answer in the cell below.**

In [19]:
answer_p_value = """ p-value singifie la probabilité d observer un échantillon avec un coefficient lińeaire égal ou supérieur
à la valeur trouvée 2252.7344153. Un intervalle de confiance de 2 sigma correspond à 95%, or t<2 donc ce test n est pas significatif à 95 % d intervalle de confiance.
"""

**❓ What are the two regressors whose effect is statistically significant at the 95% confidence level in this regression model? Store the name of the variables as a list of strings in the `significant_regressors` variable.**

In [20]:
significant_regressors = ['PERC_MARRIED_COUPLE_FAMILY','PERC_OTHER_STATE_OR_ABROAD']

**❓ Holding all other regressors constant, by how much does the total number of incidents increase or decrease when the share of families with a married couple increases by 1 percentage point in the district? Please formulate a full sentence with statistical rigor!**

In [21]:
answer_how_much_increase = """ coefficient PERC_MARRIED_COUPLE_FAMILY = -2115.1839
Si le pourcentage de couples mariés augment de 1 pourcent, le nombre d incident diminuerait de 2115.1839 (coefficient linéaire négatif).
"""

#### Limits of this regression model

You had asked the intern on the team to estimate a linear regression model so as to investigate the socio-economic determinants of crime in Boston. The results above are those that he presented. In the email he sent to you, he added:

> *You will probably notice the extremely high R-squared score of this model: I think we have an excellent fit and the results are solid* 😄

But you have very strong doubts about this regression and you think it is a perfect occasion to give some very important advice to your intern...

**❓  What is the main limitation of this (clearly spurious) regression according to you? This observation explains why we are getting a very high R-squared and large standard errors. Please provide your answer in the following Markdown cell.**

In [22]:
answer_limitations = """
R**2 = 0.899, ce qui est une valeur élevée et montre une relation linéaire entre le nombre d incidents et
les paramètres du modèle. Cependant, une valeur élevée du coefficient R**2 ne traduit pas toujours un lien de cause à
effet entre les variables.
"""

### 💾 Run the cell below to submit your results

In [23]:
from nbresult import ChallengeResult
import json
answers_inference = {"MEDIAN_AGE": answer_median_age,
                    "T_STAT":answer_t_statistic,
                     "P_VALUE": answer_p_value,
                     "INCREASE": answer_how_much_increase,
                     "LIMITATIONS": answer_limitations}

with open("tests/answers_inference.json", "w", encoding="utf-8") as f:
    json.dump(answers_inference, f, ensure_ascii=False, indent=4)

result = ChallengeResult(
    'linear_regression_analysis',
    model=model,
    numerator=numerator,
    denominator=denominator,
    t=t_median_age,
    pvalue=pvalue_median_age,
    regressors=significant_regressors
)
result.write()

# 2. Analysis for the police department (4h)

**This part assesses the last skill of the first block of the certification _Analyse exploratoire et descriptive de la donnée_:**
- C4 Concevoir une représentation graphique adaptée afin de visualiser les relations entre les données

**and the skills of the fourth block _Développement et Gestion de projet en IA_:**
- C18 Analyser et formuler une problématique métier du point de vue de l'analyste de données
- C19 Planification des actions à mettre en œuvre
- C20 Mobiliser les ressources nécessaires pour mener à bien le projet en respectant les délais
- C21 Recommander des actions à sa hiérarchie et rendre compte de ses résultats  

🚨 **The expected deliverable for this part is the Short presentation**  

The head of the Police Department of Boston, who read your report for the Mayor's team, was extremely interested in the results. He contacted your consulting firm for an additional presentation, that would focus on the nature of crimes that take place in Boston, the potential time trends that you could identify and/or the heterogeneity of the impact of criminality on the different police districts. 

## 2.1 Start with a fresh dataset

You will start from a fresh dataset, that corresponds more or less to the `incident_reports` table of the database.

In [24]:
# Run this cell to download the datasets in the data directory
#!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/incident_reports.csv > data/incident_reports.csv   
#!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/districts.csv > data/districts.csv    

In [25]:
# Load the dataset
from data import load_data_viz_data

data = load_data_viz_data()
data[['LAT','LONG']] = data[['LAT','LONG']].astype('float64')

print("Shape of the DataFrame:", data.shape)

data.head(10)

Shape of the DataFrame: (237221, 9)


,INCIDENT_NUMBER,OFFENSE_CODE_GROUP,SHOOTING,OCCURRED_ON_DATE,LAT,LONG,NAME,LAT_POLICE_STATION,LONG_POLICE_STATION
0,I192068249,Other,0,2015-08-28 10:20:00,42.330119,-71.084251,Roxbury,42.328894,-71.085359
1,I182074094,Violence and harassment,0,2015-09-14 09:31:00,42.315142,-71.067047,Roxbury,42.328894,-71.085359
2,I182054888,Violence and harassment,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
3,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
4,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
5,I172075854,Fraud and law violations,0,2015-09-02 09:00:00,42.314013,-71.086353,Roxbury,42.328894,-71.085359
6,I172065681,Fraud and law violations,0,2015-07-25 02:00:00,42.316339,-71.068800,Roxbury,42.328894,-71.085359
7,I172060902,Fraud and law violations,0,2015-11-12 12:00:00,42.316057,-71.078200,Roxbury,42.328894,-71.085359
8,I172026505,Fraud and law violations,0,2015-08-01 12:00:00,42.330013,-71.091179,Roxbury,42.328894,-71.085359
9,I172014289,Other,0,2015-12-01 12:00:00,42.323410,-71.093396,Roxbury,42.328894,-71.085359


## 2.1 Further data manipulations

In this section, we are going to answer two specific questions that the head of the Police Department of Boston asked your team, about the characteristics of the incidents that occur in the various districts of the municipality.

### 2.1.1 Most represented type of incident per district

First, the head of the Police Department of Boston wants to know what incident category is most often found in each district. 

**📝 Construct a DataFrame with**
- **one row per police district, designated by its full name**
- **one text column that indicates the name of the most common category of incident in the district over the whole sample period.**

💡 Hint: you may need to first define a custom aggregation function.

In [26]:
df_most_common = pd.DataFrame([],columns = ['DISTRICT','MOST_COMMON_INCIDENT'])
i=0
for dis in data['NAME'].unique():
    df = data[data['NAME'] == dis]
    rank = 0 #most common
    off = df[['NAME','OFFENSE_CODE_GROUP']].value_counts().index[rank][1]
    df_most_common.loc[i] = [dis,off]
    i=i+1

In [27]:
#df_most_common

**❓ Can you tell what is the second most common offense int the Brighton district?**

In [28]:
df_second_common = pd.DataFrame([],columns = ['DISTRICT','SECOND_COMMON_INCIDENT'])
i=0
for dis in data['NAME'].unique():
    df = data[data['NAME'] == dis]
    rank = 1 #most common
    off = df[['NAME','OFFENSE_CODE_GROUP']].value_counts().index[rank][1]
    df_second_common.loc[i] = [dis,off]
    i=i+1
#df_second_common

In [29]:
#D après le précédent Dataframe
second_most_common_offense = "Police investigation procedure"

### Average distance to the police station per district

Second, based on the Haversine distance function defined below, the head of the Police Department would like to know, for each district, **the average distance between the location of the incident and the police station**. 

**📝 Construct a DataFrame with one row per police district, designated by its full name, and one column that displays this average Haversine distance.**

In [30]:
# Haversine distance function
from math import radians, sin, cos, asin, sqrt

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Compute distance (km) between two pairs of (lat, lng) coordinates
    See - (https://en.wikipedia.org/wiki/Haversine_formula)
    """
    
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    
    return 2 * 6371 * asin(sqrt(a))

In [31]:
data.head()

,INCIDENT_NUMBER,OFFENSE_CODE_GROUP,SHOOTING,OCCURRED_ON_DATE,LAT,LONG,NAME,LAT_POLICE_STATION,LONG_POLICE_STATION
0,I192068249,Other,0,2015-08-28 10:20:00,42.330119,-71.084251,Roxbury,42.328894,-71.085359
1,I182074094,Violence and harassment,0,2015-09-14 09:31:00,42.315142,-71.067047,Roxbury,42.328894,-71.085359
2,I182054888,Violence and harassment,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
3,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
4,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359


In [43]:
for i in range(0, len(data)):
    #if(i%10000==0):
    #    print(i)
    #if(i==len(data)-1):
    #    print('FINISH')
    data.loc[i,'distance']=haversine_distance(data.loc[i,'LONG'], data.loc[i,'LAT'], data.loc[i,'LONG_POLICE_STATION'], data.loc[i,'LAT_POLICE_STATION'])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
FINISH


In [47]:
data

,INCIDENT_NUMBER,OFFENSE_CODE_GROUP,SHOOTING,OCCURRED_ON_DATE,LAT,LONG,NAME,LAT_POLICE_STATION,LONG_POLICE_STATION,distance
0,I192068249,Other,0,2015-08-28 10:20:00,42.330119,-71.084251,Roxbury,42.328894,-71.085359,0.163832
1,I182074094,Violence and harassment,0,2015-09-14 09:31:00,42.315142,-71.067047,Roxbury,42.328894,-71.085359,2.145919
2,I182054888,Violence and harassment,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359,2.021175
3,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359,2.021175
4,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359,2.021175
...,...,...,...,...,...,...,...,...,...,...
237216,I172049757,Police investigation procedure,0,2018-08-09 18:00:00,42.355216,-71.060129,Charlestown,42.361800,-71.060300,0.732215
237217,I172049757,Drugs and disorderly conduct,0,2018-08-09 18:00:00,42.355216,-71.060129,Charlestown,42.361800,-71.060300,0.732215
237218,I172027369,Other,0,2018-12-12 05:00:00,42.376852,-71.068873,Charlestown,42.361800,-71.060300,1.815839
237219,I172027369,Police investigation procedure,0,2018-12-12 05:00:00,42.376852,-71.068873,Charlestown,42.361800,-71.060300,1.815839


In [49]:
data2=data.groupby(by='NAME').mean()
data2

,SHOOTING,LAT,LONG,LAT_POLICE_STATION,LONG_POLICE_STATION,distance
NAME,,,,,,
Brighton,0.001741,42.337940,-71.121492,42.349300,-71.150600,3.770867
Charlestown,0.001049,42.083411,-70.591929,42.361800,-71.060300,58.758394
Dorchester,0.007270,42.291887,-71.048714,42.298068,-71.059141,3.174444
Downtown,0.000724,42.328492,-71.015711,42.361800,-71.060300,6.336657
East Boston,0.002373,42.354856,-70.994902,42.371200,-71.038700,5.568626
Hyde Park,0.003187,42.260016,-71.113105,42.256476,-71.124279,2.397613
Jamaica Plain,0.006718,42.308171,-71.094808,42.309700,-71.104600,2.585737
Mattapan,0.010483,42.284378,-71.080442,42.284800,-71.091600,1.908636
Roxbury,0.012527,42.313967,-71.071864,42.328894,-71.085359,2.989029


**❓ Can you tell what is the average distance between the police station and the offenses in the Brighton district?**

In [51]:
average_distance_km = 3.77
#Dans le district de Bright, la distance moyenne entre l incident et le district est de 3.77 km.

### 💾 Run the cell below to submit your results

In [52]:
from nbresult import ChallengeResult

result = ChallengeResult('manipulation',
                         second_most_common_offense=second_most_common_offense,
                         average_distance_km=average_distance_km)
result.write()

# 2.2 Short presentation

🚨 (REQUIRED TO VALIDATE THE CERTIFICATION)

Using the latest dataset that you loaded, your mission is now to prepare 5 slides (including a title slide) that you would present to the head of the Police Department.  
You should focus on including the output of the two "Further data manipulations" tasks in your presentation.  

⚠️  You can use any presentation editor of your choice, but **the slides must be shared either in HTML or in PDF format and saved in the current directory**

Before you get started, here are four small pieces of advice:

- to prepare your data visualisations, do not hesitate to create a separate, blank notebook; 


- pay particular attention to the readability and the clarity of your legends, titles, charts and tables; 


- the Pandas `resample` method might be useful if you want to plot time trends;


- keep in mind that you are working with real data and sometimes, data are not very talkative. Do not feel discouraged if your charts do not provide revolutionary insights: typically, an absence of trend is a substantial piece of information!

Good luck in changing Boston residents' lives!

# A word of conclusion

Congratulations for going through the exercise 🎉

If you wish to pursue your analysis at some point, note that all datasets (and many others) are publicly available online, on the [Analyze Boston](https://data.boston.gov) website.

Besides, if you are interested in the topic, you can start by reading the work of Nobel Prize laureate Gary Becker, who was the first to model crime as a rational phenomenon, similarly to an economic decision. This model, although it has limitations, marked a breakthrough in the study of crime and paved the way for many empirical studies that further analysed the socio-economic determinants of illegal activities. 

👉 [Link](https://olis.leg.state.or.us/liz/2017R1/Downloads/CommitteeMeetingDocument/125036) to download a full-text version of "Crime and Punishment: An Economic Approach" by Becker (1968)